In [3]:
import librosa
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def feature_extractor(path, frame_size=2048, hop_length=126, num_frames=10):
    mfcc_ft = list()
    genres = os.listdir(path)
    for genre in genres:
        fullpath = os.path.join(path, genre)
        for audio in os.listdir(fullpath):
            mfcc_dict = {}
            audiopath = os.path.join(fullpath, audio)
            signal, sr = librosa.load(audiopath)

            # creating frames for each audio file
            for i in range(0, min(len(signal) - frame_size + 1, num_frames * hop_length), hop_length):
                frame = signal[i : i + frame_size]  # Extract current frame

                # Further processing and feature extraction
                mfcc_features = librosa.feature.mfcc(y=frame, sr=sr)  # Mel-Frequency Cepstral Coefficients
                chroma_features = librosa.feature.chroma_stft(y=frame, sr=sr)
                zero_crossing_rate = librosa.feature.zero_crossing_rate(y=frame)
                spectral_centroid = librosa.feature.spectral_centroid(y=frame, sr=sr)
                spectral_rolloff = librosa.feature.spectral_rolloff(y=frame, sr=sr)
                chroma_energy = librosa.feature.chroma_cens(y=frame, sr=sr)  # Chroma Energy Normalized

                # Create a new dictionary for each frame
                mfcc_dict = {}
                mfcc_dict["chroma_energy"] = round(chroma_energy.mean(), 3)
                mfcc_dict["mfcc"] = round(mfcc_features.mean(),3)
                mfcc_dict["chroma_stft"] = round(chroma_features.mean(),3)
                mfcc_dict["zero_crossing_rate"] = round(zero_crossing_rate.mean(), 3)
                mfcc_dict["spectral_centroid"] = round(spectral_centroid.mean(), 3)
                mfcc_dict["spectral_rolloff"] = round(spectral_rolloff.mean(), 3)
                mfcc_dict["genre"] = genre

                # Append the dictionary to the mfcc_ft list
                mfcc_ft.append(mfcc_dict)

    return pd.DataFrame(mfcc_ft)


Extractedfile = feature_extractor("C:/Users/bukol/OneDrive/Documents/genres")

In [ ]:
Extractedfile.to_csv("Myextractedgenre.csv", index=False)

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
import warnings
warnings.filterwarnings("ignore")

# READING IN THE DATA

In [ ]:
# As your dataset is stored in "Myextractedgenre.csv"
data = pd.read_csv("/content/drive/MyDrive/Myextractedgenre.csv")


In [ ]:
data

# DATA VISUALIZATION

In [ ]:
def plot_waveform(path, num_rows=2, num_cols=5):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Define the number of rows and columns for subplots
    num_rows = min(len(folders), num_rows)
    num_cols = min(num_cols, len(folders) // num_rows)

    # Create a figure with subplots
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 6), squeeze=False)
    # Iterate over the folders and plot the waveform for each audio file
    for i, folder in enumerate(folders):
        if i >= num_rows * num_cols:
            break  # Limit the number of genres to plot based on the number of subplots
        folder_path = os.path.join(path, folder)
        j = 0  # Counter to limit the number of audio files plotted per genre
        for file in os.listdir(folder_path):
            if file.endswith('.wav') and j < num_cols:
                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)
                row = i // num_cols
                col = i % num_cols
                axs[row, col].plot(audio, linewidth=1)
                axs[row, col].set_title(folder)
                axs[row, col].set_xticks([])
                axs[row, col].set_yticks([])
                j += 1

    plt.tight_layout()
    fig.suptitle("Waveforms")
    fig.subplots_adjust(top=0.90)
    plt.savefig('waveforms_for_audio_files.png', dpi=300)
    plt.show()

plot_waveform('/content/drive/MyDrive/MUSIC', num_rows=2, num_cols=5)

In [ ]:

def plot_spectrograms(path, num_rows=2, num_cols=5):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Create a figure with subplots
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 6), squeeze=False)

    # Iterate over the folders and plot the spectrogram for each audio file
    for i, folder in enumerate(folders):
        if i >= num_rows * num_cols:
            break  # Limit the number of genres to plot based on the number of subplots

        folder_path = os.path.join(path, folder)
        for j, file in enumerate(os.listdir(folder_path)):
            if file.endswith('.wav'):
                if j >= num_cols:
                    break  # Limit the number of audio files to plot based on the number of columns

                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)

                # Compute the MFCC features for the audio file
                mfcc_features = librosa.feature.mfcc(y=audio, sr=sr)

                # Plot the MFCC spectrogram
                row = i // num_cols
                col = i % num_cols
                librosa.display.specshow(mfcc_features, sr=sr, x_axis='time', ax=axs[row, col])
                axs[row, col].set_title(f'{folder}')
                axs[row, col].set_xlabel('Time (s)')
                axs[row, col].set_ylabel('MFCC')

    plt.tight_layout()
    fig.suptitle("MFCC Spectrograms")
    fig.subplots_adjust(top=0.90)
    plt.savefig('mfcc_spectrograms.png', dpi=300)
    plt.show()

plot_spectrograms("/content/drive/MyDrive/MUSIC")


In [ ]:

def plot_spectrograms(path, num_rows=2, num_cols=5):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Create a figure with subplots
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 6), squeeze=False)

    # Iterate over the folders and plot the spectrogram for each audio file
    for i, folder in enumerate(folders):
        if i >= num_rows * num_cols:
            break  # Limit the number of genres to plot based on the number of subplots

        folder_path = os.path.join(path, folder)
        for j, file in enumerate(os.listdir(folder_path)):
            if file.endswith('.wav'):
                if j >= num_cols:
                    break  # Limit the number of audio files to plot based on the number of columns

                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)

                # Compute the chroma_energy feature for the audio file
                chroma_energy = librosa.feature.chroma_cens(y=audio, sr=sr)

                # Plot the chroma_energy spectrogram
                row = i // num_cols
                col = i % num_cols
                librosa.display.specshow(chroma_energy, sr=sr, x_axis='time', ax=axs[row, col])
                axs[row, col].set_title(f'{folder}')
                axs[row, col].set_xlabel('Time (s)')
                axs[row, col].set_ylabel('Chroma Energy')

    plt.tight_layout()
    fig.suptitle("Chroma Energy Spectrograms")
    fig.subplots_adjust(top=0.90)
    plt.savefig('chroma_energy_spectrograms.png', dpi=300)
    plt.show()

plot_spectrograms("/content/drive/MyDrive/MUSIC")


In [ ]:

def plot_spectrograms(path, num_rows=2, num_cols=5):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Create a figure with subplots
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 6), squeeze=False)

    # Iterate over the folders and plot the spectrogram for each audio file
    for i, folder in enumerate(folders):
        if i >= num_rows * num_cols:
            break  # Limit the number of genres to plot based on the number of subplots

        folder_path = os.path.join(path, folder)
        for j, file in enumerate(os.listdir(folder_path)):
            if file.endswith('.wav'):
                if j >= num_cols:
                    break  # Limit the number of audio files to plot based on the number of columns

                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)

                # Compute the chroma_stft feature for the audio file
                chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)

                # Plot the chroma_stft spectrogram
                row = i // num_cols
                col = i % num_cols
                librosa.display.specshow(chroma_stft, sr=sr, x_axis='time', ax=axs[row, col])
                axs[row, col].set_title(f'{folder}')
                axs[row, col].set_xlabel('Time (s)')
                axs[row, col].set_ylabel('Chroma STFT')

    plt.tight_layout()
    fig.suptitle("Chroma STFT Spectrograms")
    fig.subplots_adjust(top=0.90)
    plt.savefig('chroma_stft_spectrograms.png', dpi=300)
    plt.show()

plot_spectrograms("/content/drive/MyDrive/MUSIC")

In [ ]:

def plot_zero_crossing_rate(path, num_rows=2, num_cols=5):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Create a figure with subplots
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 6), squeeze=False)

    # Iterate over the folders and plot the zero crossing rate for each audio file
    for i, folder in enumerate(folders):
        if i >= num_rows * num_cols:
            break  # Limit the number of genres to plot based on the number of subplots

        folder_path = os.path.join(path, folder)
        for j, file in enumerate(os.listdir(folder_path)):
            if file.endswith('.wav'):
                if j >= num_cols:
                    break  # Limit the number of audio files to plot based on the number of columns

                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)

                # Compute the zero_crossing_rate feature for the audio file
                zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)

                # Plot the zero_crossing_rate values over time
                time = librosa.times_like(zero_crossing_rate, sr=sr)
                row = i // num_cols
                col = i % num_cols
                axs[row, col].plot(time, zero_crossing_rate[0])
                axs[row, col].set_title(f'{folder}')
                axs[row, col].set_xlabel('Time (s)')
                axs[row, col].set_ylabel('Zero Crossing Rate')

    plt.tight_layout()
    fig.suptitle("Zero Crossing Rate")
    fig.subplots_adjust(top=0.90)
    plt.savefig('zero_crossing_rate.png', dpi=300)
    plt.show()


plot_zero_crossing_rate("/content/drive/MyDrive/MUSIC")


In [ ]:

def count_zero_crossings(path):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Iterate over the folders and calculate the count of zero crossings for each audio file
    for folder in folders:
        folder_path = os.path.join(path, folder)
        for file in os.listdir(folder_path):
            if file.endswith('.wav'):
                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)

                # Calculate the count of zero crossings
                zero_crossings_count = sum(librosa.zero_crossings(audio))

                print(f"File: {file}, Genre: {folder}, Zero Crossings Count: {zero_crossings_count}")

count_zero_crossings("/content/drive/MyDrive/MUSIC")


In [ ]:

def plot_spectral_centroid(path, num_rows=2, num_cols=5):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Create a figure with subplots
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 6), squeeze=False)

    # Iterate over the folders and plot the spectral centroid for each audio file
    for i, folder in enumerate(folders):
        if i >= num_rows * num_cols:
            break  # Limit the number of genres to plot based on the number of subplots

        folder_path = os.path.join(path, folder)
        for j, file in enumerate(os.listdir(folder_path)):
            if file.endswith('.wav'):
                if j >= num_cols:
                    break  # Limit the number of audio files to plot based on the number of columns

                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)

                # Compute the spectral centroid for the audio file
                spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)

                # Plot the spectral centroid
                row = i // num_cols
                col = i % num_cols
                axs[row, col].plot(librosa.times_like(spectral_centroids, sr=sr), spectral_centroids[0])
                axs[row, col].set_title(f'{folder}')
                axs[row, col].set_xlabel('Time (s)')
                axs[row, col].set_ylabel('Spectral Centroid')

    plt.tight_layout()
    fig.suptitle("Spectral Centroid")
    fig.subplots_adjust(top=0.90)
    plt.savefig('spectral_centroid.png', dpi=300)
    plt.show()

plot_spectral_centroid("/content/drive/MyDrive/MUSIC")


In [ ]:

def plot_spectral_rolloff(path, num_rows=2, num_cols=5):
    # List all subdirectories (folders) in the specified directory
    folders = next(os.walk(path))[1]

    # Create a figure with subplots
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 6), squeeze=False)

    # Iterate over the folders and plot the spectral rolloff for each audio file
    for i, folder in enumerate(folders):
        if i >= num_rows * num_cols:
            break  # Limit the number of genres to plot based on the number of subplots

        folder_path = os.path.join(path, folder)
        for j, file in enumerate(os.listdir(folder_path)):
            if file.endswith('.wav'):
                if j >= num_cols:
                    break  # Limit the number of audio files to plot based on the number of columns

                file_path = os.path.join(folder_path, file)
                audio, sr = librosa.load(file_path)

                # Compute the spectral rolloff for the audio file
                spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)

                # Plot the spectral rolloff
                row = i // num_cols
                col = i % num_cols
                axs[row, col].plot(librosa.times_like(spectral_rolloff, sr=sr), spectral_rolloff[0])
                axs[row, col].set_title(f'{folder}')
                axs[row, col].set_xlabel('Time (s)')
                axs[row, col].set_ylabel('Spectral Rolloff')

    plt.tight_layout()
    fig.suptitle("Spectral Rolloff")
    fig.subplots_adjust(top=0.90)
    plt.savefig('spectral_rolloff.png', dpi=300)
    plt.show()

plot_spectral_rolloff("/content/drive/MyDrive/MUSIC")

# DATASET PREPARATION FOR MODEL TRAINING

In [ ]:
X= np.array(data.iloc[:,:-1],dtype='float32')
X.shape


In [ ]:
# LabelEncoding
class_list= data.genre
convertor=LabelEncoder()

y= convertor.fit_transform(class_list)
y = tf.keras.utils.to_categorical(y, num_classes=10)
y.shape

In [ ]:
def prepare_data(X,y,test_size=0.1, validation_size=0.1):

    # dividing the data into test, train sets
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=test_size)

    # dividing the Training set into Training and validation dataset
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size= validation_size)


    #Creating a new axis for the numpy array to make it a 4 d array
    X_train=X_train[..., np.newaxis]
    X_validation=X_validation[..., np.newaxis]
    X_test=X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation , y_test

In [ ]:
# SCALING THE FEATURES

#using StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)

# DEFINING THE MODEL

In [ ]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, y, test_size=0.2, random_state=42)


# Define the metrics
precision = Precision()
recall = Recall()


  # Create a TensorFlow model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', precision, recall])

# TRAINING THE MODEL

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))


# EVALUATING THE MODEL

In [ ]:
# Evaluate the model
results = model.evaluate(X_test, y_test)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])


# PLOTTING MODEL PERFORMANCE

In [ ]:

# Extract the recorded metrics from the 'history' object
epochs = range(1, len(history.history['accuracy']) + 1)
accuracy = history.history['accuracy']
loss = history.history['loss']
precision = history.history['precision']  
recall = history.history['recall']  

# Plot metrics over epochs
plt.figure(figsize=(10, 6))

plt.plot(epochs, accuracy, label='Accuracy')
plt.plot(epochs, loss, label='Loss')
plt.plot(epochs, precision, label='Precision')
plt.plot(epochs, recall, label='Recall')

plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Model Performance Metrics over Epochs')
plt.grid()
plt.legend()
plt.show()
